In [ ]:
pip install obspy 

In [ ]:
pip install future 

In [ ]:
# install the data time module to use for download and data analysis

from obspy.core import UTCDateTime
UTCDateTime("2012-09-07T12:15:00")
UTCDateTime(2012, 9, 7, 12, 15)
UTCDateTime(2012, 9, 7, 12, 15, 0)
UTCDateTime(2012, 9, 7, 12, 15)
UTCDateTime(1347020100.0)
UTCDateTime(2012, 9, 7, 12, 15)

In [ ]:
# importing useful python modules

import numpy as np
import matplotlib.pyplot as plt
import obspy
import obspy.signal
from obspy import read, read_inventory


In [ ]:
# obtaining event data for Switzerland

# "min_lat": 45.83203,
   # "max_lat": 47.69732,
   # "min_lng": 6.07544,
   # "max_lng": 9.83723

from __future__ import print_function

import obspy
import obspy.clients.fdsn


client = obspy.clients.fdsn.Client("ETH")

starttime=obspy.UTCDateTime("2001-01-01")
endtime=obspy.UTCDateTime("2020-01-01")

events = client.get_events(minlatitude=45.83203, maxlatitude=47.69732,
                           minlongitude=6.07544, maxlongitude=9.83723,
                           minmagnitude=0, maxmagnitude=1,
                           starttime=starttime,
                           endtime=endtime, catalog = 'ETH')

print("found %s event(s):" % len(events))
for event in events:
    print(event)

In [ ]:
# downloading data for switzerland


# for oklahoma the main seismic stations belong to the IRISDMC, (Link: https://www.fdsn.org/networks/detail/OK/). 
# these stations are not directly observed in obspy but are embedded with the IRIS_client.


from obspy.clients.fdsn.mass_downloader import CircularDomain, \
    Restrictions, MassDownloader

# downloading data for 40 seconds

origin_time = obspy.UTCDateTime(2007, 5, 21, 4, 19, 0.00)


# Circular domain around the epicenter. This will download all data between
# 0 and 5 degrees distance from the epicenter.

# the minimum and maxradius radi are important parameters

domain = CircularDomain(latitude=47.58, longitude=7.6,
                       
                        minradius=0.0, maxradius=5.0)

restrictions = Restrictions(

    # obtaining waveform information 10 seconds before the event and 30 seconds after
    
    starttime=origin_time - 0.1667*60,
    endtime=origin_time + 0.5*60,
    
    
    # Dealing with gaps in the data
    # True, any trace with a gap/overlap will be discarded.
    
    
    reject_channels_with_gaps=True,
    
    # obtaining only want waveforms that have data for at least 95 %  the requested time span. Any trace that is shorter than 95 % of the
    # desired total duration will be discarded.
    
    
    minimum_length=0.95,
    
    
    
    minimum_interstation_distance_in_m=0.005,
    
    # Only HH or BH channels. If a station has HH channels, those will be
    # downloaded, otherwise the BH. Nothing will be downloaded if it has
    # neither.
    
    channel_priorities=["HH[ZNE]", "BH[ZNE]"],
    
    # Location codes are arbitrary and there is no rule as to which
    # location is best. Same logic as for the previous setting.
    
    location_priorities=["", "00", "10"])

# No specified providers will result in all known ones being queried.

mdl = MassDownloader(providers=["ETH"])

# The data will be downloaded to the ``./waveforms/`` and ``./stations/``
# folders with automatically chosen file names.

mdl.download(domain, restrictions, mseed_storage="waveforms_earthquake_data_1",
             stationxml_storage="stations_earthquake_data_1")
